In [1]:
import pandas as pd
import numpy as np
import distance
import json
import os
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import psycopg2
import analytics

In [ ]:
analytics.write_key = os.environ.get('SEGMENT_KEY')

In [ ]:
try:
    with SSHTunnelForwarder(
        (os.environ.get('JOUST_IP'), 22),
        ssh_private_key="~/.ssh/id_rsa",
        ssh_username="ubuntu",
        remote_bind_address=(os.environ.get('JOUST_ADDRESS'), 5432)) as server:

        server.start()
        print("server connected")

        params = {
            "database": "joust_production",
            "user": "sid",
            "password": os.environ.get('JOUST_PASS'),
            "host": "localhost",
            "port": server.local_bind_port
        }

        conn = psycopg2.connect(**params)
        curs = conn.cursor()
        print("database connected")

        #curs.execute("select * from public.account_holders")
        transfers = "select account_holders.id, first_name, last_name, amount, transfers.created_at as \"created_at.1\", core_pro_customers.customer_id as q2_core_pro_customer_id, from_account_id, transfers.customer_id, to_account_id from account_holders inner join transfers on account_holders.id = transfers.account_holder_id left join core_pro_customers on core_pro_customers.account_holder_id = account_holders.id"
        invoices = "select account_holders.id, first_name, last_name, payment_requests.status, amount, account_holders.created_at, payment_requests.created_at as \"created_at.1\", accepted_date, core_pro_customers.customer_id as q2_core_pro_customer_id, payarmour_service, payment_type from account_holders inner join payment_requests on account_holders.id = payment_requests.account_holder_id left join core_pro_customers on core_pro_customers.account_holder_id = account_holders.id"
        total = "select account_holders.id, account_holders.segment_intercom_id, first_name, last_name, occupations.\"name\" as occupation, industries.\"name\" as industry, marketing.value as legacy_business_url, type_of_work, locations.country, account_holders.created_at, sign_in_count, ssn, email, mobile_phone, date_of_birth, expected_yearly_income_id, core_pro_customers.customer_id as q2_core_pro_customer_id, payarmour_service from account_holders left join core_pro_customers on core_pro_customers.account_holder_id = account_holders.id left join locations on account_holders.id = locations.locatable_id left join occupations on account_holders.occupation_id = occupations.id left join industries on occupations.industry_id = industries.id left join businesses on businesses.account_holder_id = account_holders.id left join marketing on marketing.business_id = businesses.id and marketing.source = 2"
        plaid = "select account_holders.id, first_name,last_name,plaid_identities.updated_at,plaid_identities.raw_response->>'owners' from account_holders inner join external_bank_accounts on account_holders.id = external_bank_accounts.account_holder_id inner join plaid_identities on external_bank_accounts.id = plaid_identities.plaid_identifiable_id and plaid_identifiable_type='ExternalBankAccount'"
        alloy = "SELECT account_holders.first_name, account_holders.last_name, account_holders.id, alloy_results.alloy_score as alloy_id FROM account_holders INNER JOIN alloy_results ON alloy_results.resultable_id = account_holders.id AND alloy_results.resultable_type = 'AccountHolder'"
        bank = "select account_holders.id, first_name,last_name,bank_accounts.q2_account_id,bank_accounts.is_primary,bank_accounts.available_balance from account_holders inner join bank_accounts on bank_accounts.account_holder_id = account_holders.id"
        devices = "select account_holders.id, first_name, last_name, devices.signature, devices.platform, devices.created_at  from account_holders inner join account_holder_devices on account_holders.id = account_holder_devices.account_holder_id inner join devices on account_holder_devices.device_id = devices.id"
        stat = "select account_holders.id, account_holders.created_at, account_holders.archived, account_activations.status, account_holders.email, first_name, last_name, verification_status as q2_core_pro_verification_status from account_holders left join core_pro_customers on account_holders.id = core_pro_customers.account_holder_id left join account_activations on account_holders.id = account_activations.account_holder_id"
        events = "SELECT events.payload -> 'payloadTypeId' AS id, events.payload -> 'data' -> 0 -> 0 -> 'returnCode' AS returncode, events.payload -> 'data' -> 0 -> 0 -> 'type' AS type, events.payload -> 'data' -> 0 -> 0 -> 'amount' AS amount, events.payload -> 'data' -> 0 -> 0 -> 'customerId' AS customerId, events.payload -> 'data' -> 0 -> 0 -> 'createdDate' AS createdDate FROM \"events\""

        #result = curs.fetchall()
        tra = pd.read_sql_query(transfers, conn)
        req = pd.read_sql_query(invoices, conn)
        ids = pd.read_sql_query(total, conn)
        ids.drop_duplicates(subset ="id", inplace = True)
        plaid = pd.read_sql_query(plaid, conn)
        bank = pd.read_sql_query(bank, conn)
        cells = pd.read_sql_query(devices, conn)
        rte = pd.read_sql_query(events, conn)
        status = pd.read_sql_query(stat, conn)

        conn.close()

except Exception as e:
    print(e)
    print("Connection Failed")

In [ ]:
try:
    params = {
        "database": "martechdb",
        "user": "sid",
        "password": os.environ.get('MARTECH_PASS'),
        "host": os.environ.get('MARTECH_ADDRESS'),
        "port": 5439
    }

    conn = psycopg2.connect(**params)
    curs = conn.cursor()
    print("database connected")
    
    negative = "select * from ruby.negative_balance_alert"

    negj = pd.read_sql_query(negative, conn)
    
    conn.close()

except Exception as e:
    print(e)
    print("Connection Failed")

In [ ]:
## As data gets big, do the query to isolate new users and new infos

In [ ]:
ids = ids.rename(columns={"first_name": "First Name", "last_name": "Last Name", "type_of_work": "Type Of Work", "created_at": "Created At", "legacy_business_url": "business_url", "sign_in_count": "Sign In Count"})
req = req.rename(columns={"first_name": "First Name", "last_name": "Last Name", "status": "Status", "amount": "Amount (Payment Requests)", "created_at": "Created At", "created_at.1": "Created At (Payment Requests)", "accepted_date": "Accepted Date", "q2_core_pro_customer_id": "Q2 Core Pro Customer Id"})
plaid = plaid.rename(columns={"first_name": "First Name", "last_name": "Last Name", "?column?": "Raw Response"})
tra = tra.rename(columns={"first_name": "First Name", "last_name": "Last Name", "amount": "Amount", "created_at.1": "Created At (Transfers)", "q2_core_pro_customer_id": "Q2 Core Pro Customer Id"})
bank = bank.rename(columns={"first_name": "First Name", "last_name": "Last Name"})
negj = negj.rename(columns={"name": "Full Name"})
cells = cells.rename(columns={"first_name": "First Name", "last_name": "Last Name"})

In [ ]:
ids = ids[['id','segment_intercom_id','First Name','Last Name','ssn','email','mobile_phone','date_of_birth','business_url','occupation','industry','country','expected_yearly_income_id','q2_core_pro_customer_id','Type Of Work','Created At','payarmour_service','Sign In Count']]
req = req[['id','First Name','Last Name','Status','Amount (Payment Requests)','Created At','Created At (Payment Requests)','Accepted Date','Q2 Core Pro Customer Id','payarmour_service','payment_type']]
tra = tra[['id','First Name','Last Name','Amount','Created At (Transfers)','Q2 Core Pro Customer Id','from_account_id','customer_id','to_account_id']]
negj = negj[['Full Name','balance','user_id','effective_date','type']]

In [ ]:
ids['Created At'] = pd.to_datetime(ids['Created At'])
req['Created At'] = pd.to_datetime(req['Created At'])
req['Created At (Payment Requests)'] = pd.to_datetime(req['Created At (Payment Requests)'])
req['Accepted Date'] = pd.to_datetime(req['Accepted Date'])
tra['Created At (Transfers)'] = pd.to_datetime(tra['Created At (Transfers)'])
negj['effective_date'] = pd.to_datetime(negj['effective_date'])
plaid['updated_at'] =  pd.to_datetime(plaid['updated_at'])

In [ ]:
ids['now'] = datetime.utcnow()
req['now'] = datetime.utcnow()
tra['now'] = datetime.utcnow()
plaid['now'] = datetime.utcnow()

In [ ]:
req['Full Name'] = req['First Name'].astype(str).values + ' ' + req['Last Name'].astype(str).values
req = req.drop(['First Name','Last Name'], axis=1)
tra['Full Name'] = tra['First Name'].astype(str).values + ' ' + tra['Last Name'].astype(str).values
tra = tra.drop(['First Name','Last Name'], axis=1)
plaid['Full Name'] = plaid['First Name'].astype(str).values + ' ' + plaid['Last Name'].astype(str).values
plaid = plaid.drop(['First Name','Last Name'], axis=1)
ids['Full Name'] = ids['First Name'].astype(str).values + ' ' + ids['Last Name'].astype(str).values
ids = ids.drop(['First Name','Last Name'], axis=1)
bank['Full Name'] = bank['First Name'].astype(str).values + ' ' + bank['Last Name'].astype(str).values
bank = bank.drop(['First Name','Last Name'], axis=1)
cells['Full Name'] = cells['First Name'].astype(str).values + ' ' + cells['Last Name'].astype(str).values
cells = cells.drop(['First Name','Last Name'], axis=1)

In [ ]:
#names = ['Lamine Zarrad', 'Kathryn Gruenefeldt', 'George Kurtyka', 'Vicki Apodaca', 'RitaBeth Crague', 'Jenaya Zarrad', 'Heather Lunde']
names = []
for n in names:
    req = req[req['Full Name'] != n]
    tra = tra[tra['Full Name'] != n]
    plaid = plaid[plaid['Full Name'] != n]
    ids = ids[ids['Full Name'] != n]
    cells = cells[cells['Full Name'] != n]
    bank = bank[bank['Full Name'] != n]

In [ ]:
tra = tra.merge(bank, how='left',left_on='id', right_on='id')
tra['dir'] = 0
tra['dir'] = (tra['q2_account_id'] == tra['to_account_id'])
print(str(len(tra[tra['dir'] == True])) + ' deposits have been made for a total of $' + str(round(sum(tra[tra['dir'] == True]['Amount'].values),2)))
print(str(len(tra[tra['dir'] == False])) + ' withdrawls have been made for a total of $' + str(round(sum(tra[tra['dir'] == False]['Amount'].values),2)))

In [ ]:
rte = rte.dropna()
rte['createddate'] = pd.to_datetime(rte['createddate'])
rte['Year'] = rte['createddate'].dt.year
rte['Month'] = rte['createddate'].dt.month
rte['Day'] = rte['createddate'].dt.day
rte['Date'] = rte["Year"].astype(str) + '-' + rte["Month"].astype(str) + '-' + rte["Day"].astype(str)
rte['Date'] = pd.to_datetime(rte['Date'])
rte.head()

In [ ]:
rcids = (rte['customerid'].values)
rcodes = (rte['returncode'].values)
ramounts = (rte['amount'].values)
rdates = (rte['Date'].values)
nsfs = pd.DataFrame(rcids, columns=['ids'])
nsfs['code'] = rcodes
nsfs['amount'] = ramounts
nsfs['date'] = rdates
nsfs['date'] = pd.to_datetime(nsfs['date'])
nsfs['now'] = datetime.now()
cptojid = dict(zip(ids['q2_core_pro_customer_id'], ids['id']))
nsfs['ids'] =  nsfs['ids'].replace(cptojid)
nsfs.head()

In [ ]:
negj['now'] = datetime.now()
negj['time diff'] = (negj['now'] - negj['effective_date']).dt.days
negj = negj[negj['time diff'] < 90]
siidtojid = dict(zip(ids['segment_intercom_id'], ids['id']))
negj['user_id'] =  negj['user_id'].replace(siidtojid)

In [ ]:
ids['timediff'] = (ids['now'] - ids['Created At']).dt.total_seconds() / 3600
req['timediff'] = (req['now'] - req['Created At (Payment Requests)']).dt.total_seconds() / 3600
tra['timediff'] = (tra['now'] - tra['Created At (Transfers)']).dt.total_seconds() / 3600
plaid['timediff'] = (plaid['now'] - plaid['updated_at']).dt.total_seconds() / 3600
nsfs['timediff'] = (nsfs['now'] - nsfs['date']).dt.total_seconds() / 3600

In [ ]:
ids2 = ids[ids['timediff'] < (ids['timediff'].max()+1)]
#ids2 = ids[ids['timediff'] < (2*24)]
ovrfullnames = ids2['id'].unique()
ovrfullnames.sort(axis=0)
len(ovrfullnames)

In [ ]:
# change non ids to ids

In [ ]:
tatr1 = []
tatr1_1 = []
tatr2 = []
tatr3 = []
tatr3_2 = []
tatr5 = []
tatr6 = []
tatr7 = []
tatr7_2 = []
tatr7_3 = []
tatr8 = []
tatr10 = []
tatr11 = []
tatr11_2 = []
tatr12 = []
tatr12_1 = []
tatr12_2 = []
tatr12_3 = []
tatr12_4 = []
tatr13 = []
tatr13_2 = []
tatr14 = []
tatr15 = []
tatr16 = []
tatr17 = []
tatr18 = []
tatr19 = []
tatr20 = []
tatr22 = []
tids = []
fullnames2 = []
segment = []
for user in ovrfullnames:
    name1 = req[req['id'] == user].sort_values('Created At (Payment Requests)', ascending=True)
    name2 = tra[tra['id'] == user].sort_values('Created At (Transfers)', ascending=True)
    name3 = plaid[plaid['id'] == user].sort_values('updated_at', ascending=False)
    name4 = ids[ids['id'] == user]
    name6 = negj[negj['user_id'] == user]
    name7 = cells[cells['id'] == user].sort_values('created_at', ascending=False)
    atr1 = len(name1[name1['Status'] == 1])
    tatr1.append(atr1) 
    atr1_1 = 0
    if (len(name1) > 0):
        atr1_1 = len(name1[name1['Status'] == 1])/len(name1)
    tatr1_1.append(atr1_1)
    name1['time since start'] = name1['Created At (Payment Requests)'] - name1['Created At']
    day1t = name1[name1['time since start'] < timedelta(days=1)]
    atr2 = len(day1t)
    tatr2.append(atr2)
    atr3 = name1['Amount (Payment Requests)'].max()
    tatr3.append(atr3)
    maxinv = 2500
    atr3_2 = 0
    if (len(name1) > 0):
        atr3_2 = len(name1[name1['Amount (Payment Requests)'] > (.95*maxinv)])/len(name1)
    tatr3_2.append(atr3_2)
    name1mod = name1.copy(deep=True)
    name1mod = name1mod[name1mod['Status'] == 1]
    name1mod['time diff'] = name1mod['Accepted Date'] - name1mod['Created At (Payment Requests)']
    day90 = name1mod[name1mod['time diff'] > timedelta(days=90)]
    atr5 = len(day90)
    tatr5.append(atr5)
    atr6 = len(nsfs[nsfs['ids'] == user])
    tatr6.append(atr6)
    atr7 = -1
    if len(name3) < 1:
        atr7 = 0
    else:
        rawres = name3['Raw Response'].values[0]
        obj = json.loads(rawres)
        bank = obj[0]['names'][0]
        real = name4['Full Name'].values[0]
        bank = bank.lower()
        real = real.lower()
        bank2 = bank.split(' ')
        if (bank2[-1] == 'llc'):
            atr7 = 0
        elif (bank2[-1] == 'jr' or bank2[-1] == 'sr' or bank2[-1] == 'jr.' or bank2[-1] == 'sr.'):
            bank = bank2[0] + ' ' + bank2[1]
            atr7 = distance.levenshtein(bank,real)
        else:
            bank = bank2[0] + ' ' + bank2[-1]
            atr7 = distance.levenshtein(bank,real)
    tatr7.append(atr7)
    atr7_2 = -1
    if len(name3) < 1:
        atr7_2 = 0
    else:
        atr7_2 = 1
    tatr7_2.append(atr7_2)
    atr7_3 = atr7 * atr7_2
    tatr7_3.append(atr7_3)
    atr8 = len(name6[name6['type'] != 'external'])
    tatr8.append(atr8)
    starttime = name4['Created At'].values[0]
    starttime = datetime.utcfromtimestamp(starttime.astype('O')/1e9)
    now = datetime.utcnow()
    acctage = (now - starttime).days
    atr10 = acctage
    tatr10.append(atr10)
    maxtra = 1500
    atr11 = name2[name2['dir'] == True]['Amount'].max()
    tatr11.append(atr11)
    atr11_2 = name2[name2['dir'] == False]['Amount'].max()
    tatr11_2.append(atr11_2)
    iden = ids[ids['Full Name'] == (ids[ids['id'] == user]['Full Name'].values[0])]
    # rewrite to not base id all off name?
    ssnamount = len(iden['ssn'].unique())
    tatr12_1.append(ssnamount)
    mailamount = len(iden['email'].unique())
    tatr12_2.append(mailamount)
    phoneamount = len(iden['mobile_phone'].unique())
    tatr12_3.append(phoneamount)
    dobamount = len(iden['date_of_birth'].unique())
    tatr12_4.append(dobamount)
    atr12 = max([ssnamount,mailamount,phoneamount]) - 1
    tatr12.append(atr12)
    atr13 = 'no info'
    if(len(name7) > 0):
        atr13 = name7['platform'].values[0]
    tatr13.append(atr13)
    atr13_2 = len(name7['signature'].unique())
    tatr13_2.append(atr13_2)
    sigs = name7['signature'].unique()
    totsigs = 0
    for k in sigs:
        totsigs = totsigs + len(cells[cells['signature'] == k])
    atr14 = totsigs - atr13_2
    tatr14.append(atr14)
    urls = name4.dropna(subset=['business_url'])
    loc = name4['country'].unique()
    atr15 = (loc[0] == 'US')
    tatr15.append(atr15)
    atr16 = len(name6[name6['type'] == 'external'])
    tatr16.append(atr16)
    atr17 = len(urls)>0
    tatr17.append(atr17)
    atr18 = name4['industry'].values[0]
    tatr18.append(atr18)
    atr19 = name4['occupation'].values[0]
    tatr19.append(atr19)
    name = name4['Full Name'].values[0]
    segid = name4['segment_intercom_id'].values[0]
    caps = sum(1 for c in name if c.isupper())
    atr20 = 1
    if (len(name.replace(" ", "")) != 0):
        atr20 = caps/len(name.replace(" ", ""))
    tatr20.append(atr20)
    atr22 = name4['expected_yearly_income_id'].values[0]
    tatr22.append(atr22)
    tid = user
    tids.append(tid)
    fullnames2.append(name)
    segment.append(segid)

In [ ]:
final = pd.DataFrame(fullnames2, columns=['Name'])
final['Successful Invoices'] = tatr1
final['Invoice Pct'] = tatr1_1
final['First 24 Hour'] = tatr2
final['Big Invoice Pct'] = tatr3_2
final['Overdue Invoices'] = tatr5
final['NSF count'] = tatr6
final['Linked Acct'] = tatr7_2
final['Linked Name'] = tatr7
final['Linked Transfer'] = tatr7_3
final['Negative Joust'] = tatr8
final['Acct Age'] = tatr10
final['Biggest Deposit'] = tatr11
final = final.fillna(0)
final['Attempts'] = tatr12
#final['Phone os'] = tatr13
final['Mult Phones'] = tatr13_2
final['Repeat Phone'] = tatr14
final['Domestic'] = tatr15
final['Negative External'] = tatr16
final['Website'] = tatr17
final = final.fillna(0)
final['Industry'] = tatr18
final = final.fillna('Other')
#final['Job'] = tatr19
final['Capitals'] = tatr20
final['Income'] = tatr22
final['id'] = tids
final = final.fillna(0)
final['segment'] = segment
final = final.dropna()

In [ ]:
final = final.set_index('Name')
final = final.replace(True, 1)
final = final.replace(False, 0)
ind_dict = {'Web Mobile & Software Dev':4, 'IT & Networking':4, 'Data Science & Analytics':6, 'Engineering & Architecture':6, 'Design & Creative':3, 'Translation':6, 'Legal':8, 'Admin Support':7, 'Sales & Marketing':4, 'Customer Service':7, 'Accounting & Consulting':8, 'Other':3}
final = final.replace({'Industry': ind_dict})

In [ ]:
final.loc[final['Successful Invoices'] > 5, 'Successful Invoices'] = 6
risk = .1
vals = np.array(final['Successful Invoices'].values)
best = 6
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Successful Invoices'] = edit
final['Successful Invoices'].unique()

In [ ]:
final.loc[final['Invoice Pct'] > .8, 'Invoice Pct'] = .8
risk = .25
vals = np.array(final['Invoice Pct'].values)
best = .8
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Invoice Pct'] = edit
final['Invoice Pct'].unique()

In [ ]:
final.loc[final['First 24 Hour'] > 3, 'First 24 Hour'] = 3
final.loc[final['First 24 Hour'] < 1, 'First 24 Hour'] = 1
risk = .5
vals = np.array(final['First 24 Hour'].values)
best = 1
worst = 3
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['First 24 Hour'] = edit
final['First 24 Hour'].unique()

In [ ]:
risk = .25
vals = np.array(final['Big Invoice Pct'].values)
best = 0
worst = 1
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Big Invoice Pct'] = edit
final['Big Invoice Pct'].unique()

In [ ]:
final.loc[final['Overdue Invoices'] > 4, 'Overdue Invoices'] = 4
risk = 1.25
vals = np.array(final['Overdue Invoices'].values)
best = 0
worst = 4
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Overdue Invoices'] = edit
final['Overdue Invoices'].unique()

In [ ]:
final.loc[final['NSF count'] > 3, 'NSF count'] = 3
risk = 1.75
vals = np.array(final['NSF count'].values)
best = 0
worst = 3
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['NSF count'] = edit
final['NSF count'].unique()

In [ ]:
risk = 1
vals = np.array(final['Linked Acct'].values)
best = 1
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Linked Acct'] = edit
final['Linked Acct'].unique()

In [ ]:
final.loc[final['Linked Name'] > 0, 'Linked Name'] = 1
risk = 1.75
vals = np.array(final['Linked Name'].values)
best = 1
worst = 0
edit = 100 - (100 * ((((vals - worst)/(best - worst))**risk)))
final['Linked Name'] = edit
final['Linked Name'].unique()

In [ ]:
final.loc[final['Linked Transfer'] > 0, 'Linked Transfer'] = 2
risk = 2
vals = np.array(final['Linked Transfer'].values)
best = 2
worst = 0
edit = 100 - (100 * ((((vals - worst)/(best - worst))**risk)))
final['Linked Transfer'] = edit
final['Linked Transfer'].unique()
# edit logic to account for plaid no info as 1

In [ ]:
final.loc[final['Negative Joust'] > 0, 'Negative Joust'] = 1
risk = 2
vals = np.array(final['Negative Joust'].values)
best = 0
worst = 1
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Negative Joust'] = edit
final['Negative Joust'].unique()
# change exists to 0-50 balance

In [ ]:
final.loc[final['Acct Age'] > 60, 'Acct Age'] = 60
risk = .75
vals = np.array(final['Acct Age'].values)
best = 60
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Acct Age'] = edit
final['Acct Age'].unique()

In [ ]:
final.loc[final['Biggest Deposit'] < 1425, 'Biggest Deposit'] = 1425
final.loc[final['Biggest Deposit'] > 1500, 'Biggest Deposit'] = 1500
risk = 1
vals = np.array(final['Biggest Deposit'].values)
best = 1425
worst = 1500
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Biggest Deposit'] = edit
final['Biggest Deposit'].unique()
# 1425 is .95*1500

In [ ]:
final.loc[final['Attempts'] > 2, 'Attempts'] = 2
risk = 1.25
vals = np.array(final['Attempts'].values)
best = 0
worst = 2
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Attempts'] = edit
final['Attempts'].unique()

In [ ]:
final['Mult Phones'] = final['Mult Phones'] - 1
final.loc[final['Mult Phones'] < 0, 'Mult Phones'] = 0
final.loc[final['Mult Phones'] > 2, 'Mult Phones'] = 2
risk = 1.5
vals = np.array(final['Mult Phones'].values)
best = 0
worst = 2
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Mult Phones'] = edit
final['Mult Phones'].unique()

In [ ]:
final.loc[final['Repeat Phone'] > 1, 'Repeat Phone'] = 1
risk = 1.25
vals = np.array(final['Repeat Phone'].values)
best = 0
worst = 1
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Repeat Phone'] = edit
final['Repeat Phone'].unique()

In [ ]:
risk = 2
vals = np.array(final['Domestic'].values)
best = 1
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Domestic'] = edit
final['Domestic'].unique()
# change exists to .5 for na

In [ ]:
final.loc[final['Negative External'] > 2, 'Negative External'] = 2
risk = 2
vals = np.array(final['Negative External'].values)
best = 0
worst = 2
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Negative External'] = edit
final['Negative External'].unique()

In [ ]:
risk = 1
vals = np.array(final['Website'].values)
best = 1
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Website'] = edit
final['Website'].unique()

In [ ]:
risk = .5
vals = np.array(final['Industry'].values)
best = 10
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Industry'] = edit
final['Industry'].unique()

In [ ]:
final.loc[final['Capitals'] < 1, 'Capitals'] = 0
risk = .75
vals = np.array(final['Capitals'].values)
best = 0
worst = 1
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Capitals'] = edit
final['Capitals'].unique()

In [ ]:
risk = .1
vals = np.array(final['Income'].values)
best = 5
worst = 0
edit = 100 * ((((vals - worst)/(best - worst))**risk))
final['Income'] = edit
final['Income'].unique()

In [ ]:
# throw in some socure

In [ ]:
setweights = [75, 100, 25, 50, 75, 100, 50, 100, 100, 100, 75, 75, 50, 75, 100, 100, 100, 25, 75, 100, 75]
cols = final.columns.values
setweights = np.array(setweights) / sum(setweights)
final['joust_score'] = 0
for k in range(len(setweights)):
    final['joust_score'] = final['joust_score'].values + (setweights[k] * final[cols[k]].values)

In [ ]:
#si = final['joust_score'].values
#succs = np.interp(si, (49.230769230769226, 100), (0, 100))
#final['joust_score'] = succs

In [ ]:
fin2 = final.sort_values('id', ascending=True)
fin2['Fname'] = fin2.index

In [ ]:
mydict = fin2.head().to_dict('records')[0]
type({k: mydict[k] for k in list(mydict)[:-4]})

In [ ]:
total2 = status[status['archived'] == False]
total2 = total2[total2['status'] == 2]
total2 = total2[total2['q2_core_pro_verification_status'] == 'verified']
total2.drop_duplicates(subset ="id", inplace = True)
total2 = total2.sort_values(by='id', ascending = True)
cleans = total2['id'].values
fin2 = fin2[fin2['id'].isin(cleans)]

In [ ]:
for k in fin2.to_dict('records'):
    analytics.track(k['segment'], 'update account holder score' ,{
        'attributes': {n: k[n] for n in list(k)[:-4]},
        'type': 'account_holder', 
        'id': k['id'],
        'joust_score': k['joust_score'],
        'segment': k['segment']
    })